In [1]:
from autograd import grad
import autograd.numpy as np
import matplotlib.pyplot as plt
from lista import LISTA
from functions import lasso_loss, logreg_loss
from scipy.linalg import hadamard

In [2]:
n_samples = 1000
n_test = 100
n = 10
p = 8
model = 'logreg'

loss = {'lasso': lasso_loss, 'logreg': logreg_loss}[model]

H = hadamard(p) / np.sqrt(p)
#S = p / np.arange(1, p + 1)
#S **= 2
S = np.ones(p)
K = np.dot(H, S[:, None] * H.T)

z_true = np.random.randn(p, n_samples)
D = np.random.randn(n, p).dot(K)
D /= np.linalg.norm(D, axis=0, keepdims=True)

sigma = 0.1
eps = np.random.randn(n, n_samples)

z_test = np.random.randn(p, n_test)
eps_test = np.random.randn(n, n_test)
X = np.dot(D, z_true) + eps
X_test = np.dot(D, z_test) + eps_test
if model == 'logreg':
    X = 2 * (X > 0) - 1
    X_test = 2 * (X_test > 0) - 1
lbda_max = np.max(np.abs(D.T.dot(X)))
lbda = lbda_max * 0.1
print(X.shape)

(10, 1000)


In [3]:
ista = LISTA(D, lbda, 1000, model=model)
print(loss(np.zeros((p, n_samples)), X, D, lbda))
f_star = loss(ista.transform(X), X, D, lbda)
print(f_star)

6.931471805599452
5.306823453116015


In [4]:
n_layers = 10
lista = LISTA(D, lbda, n_layers, model=model, l_star=f_star)
loss(lista.transform(X), X, D, lbda)

5.315261285250276

In [5]:
lista.fit(X, l_rate=1e-1, max_iter=1000, verbose=True)

it 0, loss = 5.315e+00, grad = 2.99e-02
it 100, loss = 5.312e+00, grad = 8.87e-03
it 200, loss = 5.312e+00, grad = 6.05e-03
it 300, loss = 5.312e+00, grad = 5.58e-03
it 400, loss = 5.312e+00, grad = 5.47e-03
it 500, loss = 5.312e+00, grad = 5.40e-03
it 600, loss = 5.312e+00, grad = 5.38e-03
it 700, loss = 5.312e+00, grad = 5.33e-03
it 800, loss = 5.312e+00, grad = 5.32e-03
it 900, loss = 5.312e+00, grad = 5.32e-03


In [10]:
lista_one = LISTA(D, lbda, 1, model=model)
lista_one.fit(X, l_rate=1e-2, max_iter=1000, batch_size=1000, verbose=True)

it 0, loss = 5.749e+00, grad = 1.76e-01
it 100, loss = 5.720e+00, grad = 1.64e-01
it 200, loss = 5.694e+00, grad = 1.54e-01
it 300, loss = 5.672e+00, grad = 1.45e-01
it 400, loss = 5.652e+00, grad = 1.37e-01
it 500, loss = 5.635e+00, grad = 1.29e-01
it 600, loss = 5.619e+00, grad = 1.22e-01
it 700, loss = 5.605e+00, grad = 1.15e-01
it 800, loss = 5.592e+00, grad = 1.10e-01
it 900, loss = 5.581e+00, grad = 1.04e-01


In [7]:
f_star = loss(LISTA(D, lbda, 1000, model=model).transform(X_test), X_test, D, lbda)
f0 = loss(np.zeros_like(z_test), X_test, D, lbda)

In [8]:
ista = LISTA(D, lbda, n_layers, model=model)
print('Avg test loss using ISTA with %d iterations: %.2e' %
      (n_layers, (loss(ista.transform(X_test), X_test, D, lbda) - f_star) / (f0 - f_star)))

Avg test loss using ISTA with 10 iterations: 5.70e-03


In [9]:
print('Avg test loss using LISTA with %d layers: %.2e' %
      (n_layers, (loss(lista.transform(X_test), X_test, D, lbda) - f_star) / (f0 - f_star)))

Avg test loss using LISTA with 10 layers: 3.51e-03
